# Дашборды с dash

In [ ]:
!pip install dash
!pip install jupyter-dash

In [ ]:
import dash
dash.__version__ # 2.17.0
#!pip install --upgrade dash --user # раскомментируйте, если у вас версия ниже

## Макет приложения Dash

Приложение Dash обычно состоит из двух частей. Первая часть - это макет, который описывает, как будет выглядеть приложение, а вторая часть описывает интерактивность. Dash предоставляет классы HTML, которые позволяют нам генерировать HTML-контент с помощью Python. Чтобы использовать эти классы, нам нужно импортировать html и dcc (dash core components).

Мы создаем новый дашборд, вызывая класс Dash(). После этого мы можем создать макет для нашего приложения. Мы используем класс Div для создания div-контейнера. Затем мы используем компоненты HTML для создания компонентов HTML, таких как H1, H2 (заголовки первого уровня, второго уровня), P (абзац) и т. д. dash имеет все теги HTML. 

Чтобы создать график на макете, мы используем класс Graph(). Graph отображает интерактивные визуализации данных с помощью plotly.js.

In [ ]:
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd

# инициализируем приложение
app = Dash()

# словарь цветов, которые будем использовать дальше
colors = {
    'background': '#ccaea9',
    'text': '#1c1238'
}

# открываем датасет
forest = pd.read_csv('populations.txt', sep='\t')

# отрисовываем столбчатую диаграмму
fig = px.bar(forest, x="year", y="lynx")

# делаем красиво
fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

# создаем макет по шаблону HTML
app.layout = html.Div([
    # заголовок первого уровня
    html.H1('Hello Dash World!', style={'textAlign': 'center', 'color': '#7FDBFF'}),
    # просто текст
    html.Div("Делаем первый дашборд в Dash"),
    # график
    dcc.Graph(
        figure=fig
    )
])

# запускаем локальный сервер
app.run_server(debug=True, port=8050, use_reloader=False)

Последняя строка запустит новый веб-сервер по адресу http://127.0.0.1:8050/. Убить сервер – нажать "Перезапустить ядро". Альтернативный вариант – создать файл .py (например, в PyCharm или VS Code), при запуске будет создаваться веб-сервер, при нажатии CTRL+C он будет убиваться.

Расположим график мультивариативного рассеивания, который делали вчера.

In [ ]:
crimes = pd.read_csv('crimeRatesByState2005.tsv', sep='\t')

In [ ]:
import plotly
import plotly.graph_objects as go

# это просто график plotly

trace0 = go.Scatter(
    x = crimes['murder'],
    y = crimes['burglary'],
    mode = 'markers',
    marker = dict(size = crimes['population']/500000,
                color = crimes['motor_vehicle_theft'],
                opacity = 0.7,
                colorscale ='Electric',
                showscale =True),
    text = crimes['state'],
    customdata = crimes['population'],
    hovertemplate =
    '<b>%{text}</b>' +
    '<br><i>Murders per capita</i>: %{x}' +
    '<br><i>Burglary per capita</i>: %{y}' +
    '<br><i>Motor Vehicle Theft per capita</i>: %{marker.color}' +
    '<br><i>Population</i>: %{customdata}',
    name='crimes'
    ) # Показатели, которые мы уложим в описание каждой точки

layout= go.Layout(
    title= 'Crime in the USA',
    hovermode= 'closest', 
    xaxis= dict(
        title= 'Murder rate (number per 100,000 population)',
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Burglary rate (number per 100,000 population)',
        gridwidth= 2,
    )
)

fig = go.Figure(data = [trace0], layout = layout)
fig


In [ ]:
# а это дашборд на веб-сервере

from dash import Dash, html, dcc
import plotly.express as px
import plotly
import plotly.graph_objects as go
import pandas as pd

app = Dash()

crimes = pd.read_csv('crimeRatesByState2005.tsv', sep='\t')

fig = go.Figure(go.Scatter(
    x = crimes['murder'],
    y = crimes['burglary'],
    mode = 'markers',
    marker = dict(size = crimes['population']/500000,
                color = crimes['motor_vehicle_theft'],
                opacity = 0.7,
                colorscale ='Electric',
                showscale =True),
    text = crimes['state'],
    customdata = crimes['population'],
    hovertemplate =
    '<b>%{text}</b>' +
    '<br><i>Murders per capita</i>: %{x}' +
    '<br><i>Burglary per capita</i>: %{y}' +
    '<br><i>Motor Vehicle Theft per capita</i>: %{marker.color}' +
    '<br><i>Population</i>: %{customdata}',
    name='crimes'
    ))

fig.update_layout(go.Layout(
    title= 'Crime in the USA',
    hovermode= 'closest', 
    xaxis= dict(
        title= 'Murder rate (number per 100,000 population)',
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Burglary rate (number per 100,000 population)',
        gridwidth= 2,
    )
))

app.layout = html.Div(children = [
    html.H1('Мультивариативный график', 
                                 style={
                                     'textAlign': 'center', 
                                     'color': colors['text']}),
    dcc.Graph(
        figure=fig,
    )])

app.run_server(debug=True, port=8050, use_reloader=False)

## Callback

Мы явно хотим иметь всякие поля для ввода, селекторы, и их каким-то образом отрабатывать. Это необходимо отдельно прописывать с помощью callback (по существу, это функции, которые будут отрабаывать изменения через наши запросы). Кстати, телеграм-боты тоже работают по такому принципу.

In [ ]:
from dash import Dash, dcc, html, Input, Output

# Часть с отрисовыванием

app = Dash()

app.layout = html.Div([
    # заголовок 6 уровня
    html.H6("Введите новое значение, и оно отобразится!"),
    # контейнер div c Input
    html.Div([
        "Введите текст: ",
        dcc.Input(id='my-input', value='начните писать текст', type='text')
    ]),
    # пустая строка
    html.Br(),
    # пустой контейнер для вывода данных
    html.Div(id='my-output'),

])

# Наш Callback

@app.callback(
    Output(component_id='my-output', component_property='children'), ## Куда выводить
    Input(component_id='my-input', component_property='value') ## Откуда брать
)

# функция, которая выведет Input в Output
def update_output_div(input_value):
    return f'Вывод: {input_value}' # Мы вовзращаем некоторую строку, которая будет вставлена в компонент с id = my-output

app.run_server(debug=True, port=8050, use_reloader=False)

Но в данном случае все достаточно просто. Мы никаким образом не меняли ничего в данных etc. Давайте попробуем сделать все то же самое, но только уже с данными:

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

app = Dash()

app.layout = html.Div([
    html.H1('ВВП и ожидаемая продолжительность жизни', style={
                                            'color': '#00008B', 
                                            'fontSize': 25,
                                            'font-family': 'Arial',
                                            'textAlign': 'center'}),
    
    # здесь потом появится график
    dcc.Graph(id='graph-with-slider'),
    
    # здесь сразу есть слайдер по годам
    dcc.Slider(
        df['year'].min(),
        df['year'].max(),
        step=None,
        value=df['year'].min(),
        marks={str(year): str(year) for year in df['year'].unique()},
        id='year-slider'
    )
])

@app.callback(
    Output('graph-with-slider', 'figure'), # говорим, что меняем фигуру (то есть прорисовываем ее заново)
    Input('year-slider', 'value')) # какие значения забираем

# функция, которая все отработает
def update_figure(selected_year):
    # делаем фильтрацию по году из слайдера
    filtered_df = df[df.year == selected_year]

    # рисуем график рассеяния
    fig = px.scatter(filtered_df, x="gdpPercap", y="lifeExp",
                     size="pop", color="continent", hover_name="country",
                     log_x=True, size_max=55)
    # задаем скорость изменения в миллисекундах
    fig.update_layout(transition_duration=500)
    # возвращаем фигуру
    return fig

app.run_server(debug=True, port=8050, use_reloader=False)

Если у нас несколько селекторов, то надо прописывать несколько input на каждое значение. Логика простая:

* есть данные, нам их надо поменять

* забираем изменения через Input (аргументы идут в порядке вызова, Input и меняем. Аналогично с Output - перечисляем и возвращаем в том же порядке, в котором давали Output)

Как можно понять, здесь уже можно спокойно делать несколько разных фильтров на один и тот же график

## Callback от других графиков

А теперь допустим, что я хочу что-то выбрать на одном графике и чтобы остальные графики обновлялись при этом (допустим, тыкаем на точку, чтобы получить информацию на других графиках)

Тоже можно!

In [ ]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px

# можно подгружать уже готовые каскадные стили для вашего дашборда
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(external_stylesheets=external_stylesheets)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')

app.layout = html.Div([
    html.Div([
        # выпадающее меню выбора страны
        dcc.Dropdown(
                df['Country Name'].unique(),
                'Russian Federation',
                id='country-selection',
            ),

        html.Div([
            # выпадающее меню выбора индикатора для диаграммы рассеяния
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'CO2 emissions (metric tons per capita)',
                id='crossfilter-xaxis-column',
            ),
            # переключатель "линейный / логарифмированный" для диаграммы рассеяния
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-xaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ],
        # стили прописываются в стиле CSS    
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            # выпадающее меню выбора страны для линейного графика
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'GDP growth (annual %)',
                id='crossfilter-yaxis-column'
            ),
            # переключатель "линейный / логарифмированный" для линейного графика
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={'padding': '10px 5px'}),

        html.Div(
            # отрисовка диаграммы рассеяния (появится после выборов во всех интерактивных элементах)
            dcc.Graph(
                id='crossfilter-indicator-scatter'
            ), style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    
        html.Div([
            # отрисовка двух линейных диаграмм
            dcc.Graph(id='x-time-series'),
            dcc.Graph(id='y-time-series'),
        ], style={'display': 'inline-block', 'width': '49%'}),
        
        # создание слайдера внизу по годам
        html.Div(dcc.Slider(
            df['Year'].min(),
            df['Year'].max(),
            step=None,
            id='crossfilter-year--slider',
            value=df['Year'].max(),
            marks={str(year): str(year) for year in df['Year'].unique()}
        ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@app.callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input('crossfilter-year--slider', 'value'))

# функция рисует график
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]
    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
            )
    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])
    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')
    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')
    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')
    return fig

# функция рисует линейный график
def create_time_series(dff, axis_type, title):
    fig = px.scatter(dff, x='Year', y='Value')
    fig.update_traces(mode='lines+markers')
    fig.update_xaxes(showgrid=False)
    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')
    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)
    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})
    return fig


@app.callback(
    Output('x-time-series', 'figure'),
    Input('country-selection', 'value'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'))

# обновляем информацию об осях X для отрисовки линейного графика
def update_y_timeseries(country, xaxis_column_name, axis_type):
    country_name = country
    dff = df[df['Country Name'] == country]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    Output('y-time-series', 'figure'),
    Input('country-selection', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-yaxis-type', 'value'))

# обновляем информацию об осях Y для отрисовки линейного графика
def update_x_timeseries(country, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == country]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

app.run_server(debug=True, port=8050, use_reloader=False)

## Задание

Сделаем дашборд по данным о чаевых (tips.csv).

Наверху должен быть селектор категориальных колонок. Далее диаграмма рассеяния – по оси X сумма счета, по оси Y сумма чаевых. Цвет точек – выбранный селектор (например, если выбрана колонка Smoker, то точки должны покраситься в два цвета – курящие и некурящие). Внизу графика должен быть слайдер с количеством гостей.

<img src="https://i.ibb.co/QMZhQDx/2024-04-10-01-18-02.png">

In [ ]:
# заготовка с импортом библиотек, датасета и внешних стилей

from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px

tips = pd.read_csv('tips.csv')

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(external_stylesheets=external_stylesheets)


###-------- СОЗДАЙТЕ LAYOUT ВАШЕГО ПРИЛОЖЕНИЯ --------###
# выпадающее меню с названиями категориальных колонок
# график
# слайдер по значениям столбца 'size'


###-------- СОЗДАЙТЕ CALLBACK --------###
# Output – график
# Input – выпадающее меню
# Input – слайдер


###-------- СОЗДАЙТЕ ФУНКЦИЮ draw_scatter --------###
# отфильтруйте датафрейм по выбранному кол-ву гостей из слайдера, сохраните в новую переменную
# создайте px.scatter с параметрами: data_frame = новый датафрейм, x = сумма счета, y = сумма чаевых,
#                                    color = цвет по выбранному параметру из выпадающего меню, 
#                                    opacity = прозрачность точек
# сохраните полученный график в переменную fig
# обновите маркеры с помощью метода fig.update_traces(), внутри пропишите marker={<словарь с параметрами>}
# обновите положение легенды с помощью fig.update_layout(legend={'orientation': "h"})
# функция должна вернуть отрисованный график

# запустите работу веб-сервера